# Digit recognition
By Cormac Raftery(G00348802)

# Contents


- Introduction 
- Folder Structure
- How To Run
- How The Program Works
    - Home Page
    - Post Page
    - Prepare The Data
    - Build model
- References

## Introduction

The goal of this repository is to create, document, and train a model that recognises hand-written digits
using the MNIST dataset.
Create a web application that allows a user to draw a digit using
their mouse or touchscreen device. The drawing should then be then be submitted
for recognition to the model you have trained above. This should be done using
the flask Python package.

## Folder Structure

Upon cloning the repository you should see the FlaskApp.py and DigitRecognition.py scripts. There should also be a folder for the created model and a static folder for the web app Draw.html.

## How To Run

To run this programme, there are a few things you must set up. You must have:

Anaconda installed -> Install anaconda
Install tensorflow -> Open an administrator cmd and type "conda install tensorflow"
Install opencv -> Open an administrator cmd and type "conda install opencv"
Install keras -> Open an administrator cmd and type "conda install keras"

It is important to run all commands in an Administrator Command Prompt

To run this program, you must open command prompt in the directory with FlaskApp.py and input the following:

python FlaskApp.py

You must then open a web browser and go to localhost:5000

## How the programme works



### Home Page

The flask app pushes the draw page in the static folder to the localhost and waits for prediction button to be pressed

In [1]:
@app.route('/')
def homePage():
    return app.send_static_file('Draw.html')

NameError: name 'app' is not defined

### Post Page

Upon the prediction button being pressed the post method searches for a bit 64 image and converts the image to black and white, then converts it to an array and sends it to DigitRecognition.py to be compared against the model

In [ ]:
@app.route('/', methods=['POST'])
def getImage():
    #Retrieving Base64 image data
    imgBase64 = request.values['imageBase64']

    #Strips the image of unnecessary contents(similar  to PIL vid)
    imgStripped = re.sub('^data:image/.+;base64,', '', imgBase64)

    #Decodes the image from Base64
    imgDecoded = base64.b64decode(imgStripped)

    #Converting the decoded image to bytes
    img = Image.open(BytesIO(imgDecoded))

    #Convert the image to grayscale
    img = img.convert('L')

    #Resizes the image to the required size of 28x28 using Bilinear mode
    img = img.resize((28, 28), Image.ANTIALIAS)
    
    #converts image to black and white 
    img = img.point(lambda p: p > 0 and 255)  

    #Convert the image to an array
    myArray = np.ndarray.flatten(np.array(img))

    #reshape the array to a 1d array
    myArray = myArray.reshape(1,784)

    #convert to type float32
    myArray = myArray.astype('float32')
    
    #divide all floats in the array so that they either have a value of 1 or 0
    myArray /= 255
    
     #Prints out the image in console
    counter = 0
    for i in myArray[0]:
        if i == 1:
           print("0",end="")
        else:
           print(".",end="")
        counter +=1

        if counter == 28:
            print("\n")
            counter = 0

    #Calls function prediction in DigitRecognition.py
    myPrediction = prediction(myArray)

    #Prints Prediction
    print("The number mostly resembles a: ")
    print(myPrediction)

    #Sends prediction back to the user.
    return str(myPrediction)

### Prepare the Data

The data from the MNIST data set must be loaded into memory before it can be used. Once loaded the data must be reshaped to size 784(28*28) as that is the size of our image. We must also convert the set to a binary representation so it can be used to calculate the loss on the model.

In [ ]:
    # load data
    (x_train,y_train), (x_test,y_test) = mnist.load_data()

    #reshape the array to a 1d array
    x_test = x_test.reshape(x_test.shape[0], 784)
    x_train = x_train.reshape(x_train.shape[0], 784)

    #set array types to  float32
    x_test = x_test.astype('float32')
    x_train = x_train.astype('float32')

    #convert the array to black and white
    x_test /= 255
    x_train /= 255

    #Adds layers for 10 classes
    y_test = keras.utils.to_categorical(y_test, 10)
    y_train = keras.utils.to_categorical(y_train, 10)

### Build Model

The model is now being created using Sequential as it allows us to use a linear stack of layers. This is created from 3 dense nerural network layers. The input shape is 784 while the output shape is 16. Once the model has been built it can then be compared  against.

In [ ]:
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=784, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    #compile the model
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

    #fit model to the training data
    model.fit(x_train, y_train, batch_size=100, epochs=10, verbose=1)

    #save the model
    model.save("models/model")


## References

https://www.python.org/

https://keras.io/

http://yann.lecun.com/exdb/mnist/

https://nextjournal.com/gkoehler/digit-recognition-with-keras

https://api.jquery.com/jQuery.post/

http://www.williammalone.com/articles/create-html5-canvas-javascript-drawing-app/

https://www.fullstackpython.com/flask.html

https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed